In [2]:
import numpy as np
import time
import csv
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.svm import SVC
from sklearn import metrics
from PIL import Image

# Data Preprocessing
Before building our model, we need to transform our dataset of 81,000 200x200 images in these ways:
1. Convert to grayscale
2. Normalize the exposure/brightness
3. Resize to 75x75
4. Extract the edges

In [ ]:
letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','nothing']

transformed_data = 'transformed_data.csv'
img_dim = 75

with open(transformed_data, newline='', mode='w') as file:
    file_writer = csv.writer(file, delimiter=',')
    for i in range(len(letters)):
        for j in range(0, 3000):
            # Pillow image module used to open file and convert to grayscale
            img = Image.open('../asl_alphabet_train/'+str(letters[i])+'+/'+str(letters[i])+str(j+1)+'+.jpg').convert('L')
            brightness = (ImageStat.Stat(img)).rms[0]

            if brightness > 130:
                img_bright = img.point(lambda p: p * 0.8)

            elif brightness <= 130:
                    img_bright = img.point(lambda p: p * 2.0)

            img_bright.show()
            img = np.asarray(img_bright)
            upper_thresh = 25
            lower_thresh = 20

            edges = cv2.Canny(img, lower_thresh, upper_thresh)
            res = cv2.resize(edges, dsizde=(img_dim,img_dim), interpolation=cv2.INTER_CUBIC)
            imgArray = np.array(res)
            flattened = np.ravel(imgArray)
            incLabel = np.append(i, flattened)

            file_writer.writerow(incLabel)


### Note: transformed_data.csv will come out to a little over 1G in size.

Shuffle the dataset to promote model integrity down the line.

In [3]:
dataset = pd.read_csv('../transformed_data.csv')
dataset = dataset.iloc[np.random.permutation(len(dataset))]

In [4]:
print("Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, 1:], dataset.iloc[:,0],
                                                    train_size=0.75, test_size=0.25)
print("X_train: {0}, X_test: {1}, Y_train: {2}, Y_test{3}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))


Splitting dataset...
X_train: (60749, 5625), X_test: (20250, 5625), Y_train: (60749,), Y_test(20250,)


Principal Components discovered after running pca = PCA(.85) on a subset of the dataset. Grid search and cross validation was used to help us determine the best parameters to test with.

Processing a dataset as large as this can be very memory intensive. Incremental PCA allows us to find our components without crashing when our resources are limited. To test the fidelity of the models, we can pickle our trained files to test on different machines.


In [5]:
print('Transforming with PCA...')
pca = IncrementalPCA(n_components=3000)

start = time.time()
pca.fit(X_train)
X_trainsformed = pca.transform(X_train)
X_testformed = pca.transform(X_test)
end = time.time()
print('X PCA Train: {0} X PCA Test: {1}'.format(X_trainsformed.shape, X_testformed.shape))
print('Time elapsed: {} min'.format(round((end-start)/60, 2)))

pca_pkl = open('pca_ASL.pkl', 'wb')
pickle.dump(pca, pca_pkl)
pca_pkl.close()

Transforming with PCA...
X PCA Train: (60749, 3000) X PCA Test: (20250, 3000)
Time elapsed: 7.11 min


In [6]:
print('Fitting SVM...')
clf = SVC(C=1e-07, kernel='linear')
start2 = time.time()
clf.fit(X_trainsformed, y_train)
end2 = time.time()
print('Time elapsed (SVM): {} min\n'.format(round((end2-start2)/60, 2)))

svm_pkl = open('svm_ASL.pkl', 'wb')
pickle.dump(clf, svm_pkl)
svm_pkl.close()

Fitting SVM...
Time elapsed (SVM): 59.32 min



In [8]:
y_pred = clf.predict(X_testformed)
print('Score: ', clf.score(X_testformed, y_test))
print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))


Score:  0.8699753086419753
Accuracy: 0.8699753086419753
